# Pre-processing (state / organization Name) Regulatory data for WaDE upload.
- Purpose:  To preprocess the data into one master file for simple DataFrame creation and extraction.

In [ ]:
# Needed Libraries / Modules

# ---- working with data ----
import os  # native operating system interaction
import numpy as np  # mathematical array manipulation
import pandas as pd  # data structure and data analysis
import geopandas as gpd  # geo-data structure and data analysis

# ---- visualization ----
import matplotlib.pyplot as plt  # plotting library
import seaborn as sns  # plotting library

# ---- API data retrieval ----
import requests  # http requests
import json  # JSON parse

# ---- Cleanup ----
import re  # string regular expression manipulation
from datetime import datetime  # date and time manipulation
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.set_option('display.float_format', lambda x: '%.5f' % x)  # suppress scientific notation in Pandas

In [ ]:
# ---- working directory ----
workingDirString = os.getcwd() # set working directory folder string here
os.chdir(workingDirString)
print(f'The working Directory is:', workingDirString)

## Regulatory Area Data #1

In [ ]:
# Input File
inputFile = "RawInputData/shapefiles/South_Dakota_Water_Development_Districts.zip"
dfin1 = gpd.read_file(inputFile).replace(np.nan, "")

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfin1:
    dfin1['WaDEUUID'] = "reSD" + dfin1.index.astype(str)
    dfin1.to_csv('RawInputData/South_Dakota_Water_Development_Districts.zip', compression=dict(method='zip', archive_name='South_Dakota_Water_Development_Districts.csv'), index=False)

print(len(dfin1))
dfin1.head()

In [ ]:
# include name of each district

name = []
for x in dfin1["WDDistrict"]:
    if x == "ED":
        name.append("East Dakota")
    elif x == "JR":
        name.append("James River") 
    elif x == "WD":
        name.append("West Dakota")
    elif x == "WR":
        name.append("West River")
    elif x == "CP":
        name.append("Central Plains")
    elif x == "SC":
        name.append("South Central")
    else:
        name.append("Vermillion Basin")
    
dfin1["name"] = name

In [ ]:
# combine polygons into districts
newdf = dfin1.dissolve(by = "name").reset_index()
newdf

In [ ]:
# create output for Regulatory Area #1 dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = newdf['WaDEUUID']

# Date Info
df['in_Date'] = "8/7/2023"
df['in_Year'] = "2023"

# Organization Info
df['in_OrganizationUUID'] = "SDre_01"

# ReportingUnit Info
df['in_EPSGCodeCV'] = "4326"
df['in_ReportingUnitName'] = newdf["name"]
df['in_ReportingUnitNativeID'] = "SD" + newdf["OBJECTID"].replace("", 0).fillna(0).astype(int).astype(str)
df['in_ReportingUnitProductVersion'] = ""
df['in_ReportingUnitTypeCV'] = "Water Development District"
df['in_ReportingUnitUpdateDate'] = "3/21/2021"
df['in_StateCV'] = "SD"
df['in_Geometry'] = ""

# Overlay Info
df['in_OversightAgency'] = "South Dakota Association of Rural Water Systems"
df['in_OverlayDescription'] = "work to promote the conservation, development and proper management of water resources"
df['in_OverlayName'] = newdf["name"]
df['in_OverlayNativeID'] = newdf["OBJECTID"]
df['in_OverlayStatusCV'] = "Active"
df['in_Statute'] = ""
df['in_StatuteLink'] =""
df['in_StatutoryEffectiveDate'] = "1/1/2011"
df['in_StatutoryEndDate'] = ""
df['in_OverlayTypeCV'] = "Water Development District"
df['in_WaterSourceTypeCV'] = "Surface Water"

df = df.drop_duplicates().reset_index(drop=True)
outdf1 = df.copy()
print(len(outdf1))
outdf1.head()

## Regulatory Area #2

In [ ]:
# etc etc,

## Concatenate POD and POU Data.  Make needed changes

In [ ]:
# Concatenate dataframes
frames = [outdf1] # list all out dataframes here
outdf = pd.concat(frames)
outdf = outdf.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(outdf))

## Clean Data / data types

In [ ]:
# Ensure Empty String / remove string value of "nan"

def ensureEmptyString(val):
    val = str(val).strip()
    if val == "" or val == " " or val == "nan" or pd.isnull(val):
        outString = ""
    else:
        outString = val
    return outString

In [ ]:
outdf['in_ReportingUnitName'] = df.apply(lambda row: ensureEmptyString(row['in_ReportingUnitName']), axis=1)
outdf['in_ReportingUnitName'].unique()

In [ ]:
outdf['in_OverlayDescription'] = df.apply(lambda row: ensureEmptyString(row['in_OverlayDescription']), axis=1)
outdf['in_OverlayDescription'].unique()

In [ ]:
outdf['in_OverlayName'] = df.apply(lambda row: ensureEmptyString(row['in_OverlayName']), axis=1)
outdf['in_OverlayName'].unique()

In [ ]:
#Update datatype of StatutoryEffectiveDate to fit WaDE 2.0 structure
outdf['in_StatutoryEffectiveDate'] = pd.to_datetime(outdf['in_StatutoryEffectiveDate'], errors = 'coerce')
outdf['in_StatutoryEffectiveDate'] = pd.to_datetime(outdf['in_StatutoryEffectiveDate'].dt.strftime('%m/%d/%Y'))
outdf['in_StatutoryEffectiveDate'].unique()

# Shapefile Data
- For attaching geometry to reporting unit info.

#### Regulatory Area #1 shapefile info

In [ ]:
# Input File
shapeInputFile = "RawInputData/shapefiles/South_Dakota_Water_Development_Districts.zip"
gdfin1 = gpd.read_file(shapeInputFile)
print(len(gdfin1))
gdfin1.head()

In [ ]:
# plot shape info to map
gdfin1.plot()

In [ ]:
# include name of each district

name = []
for x in gdfin1["WDDistrict"]:
    if x == "ED":
        name.append("East Dakota")
    elif x == "JR":
        name.append("James River") 
    elif x == "WD":
        name.append("West Dakota")
    elif x == "WR":
        name.append("West River")
    elif x == "CP":
        name.append("Central Plains")
    elif x == "SC":
        name.append("South Central")
    else:
        name.append("Vermillion Basin")
    
gdfin1["name"] = name

In [ ]:
# combine polygons into districts
newgdf = gdfin1.dissolve(by = "name").reset_index()
newgdf

In [ ]:
# plot new shape info to map

newgdf.plot()

In [ ]:
# create output for Regulatory Area #1 dataframe
df = pd.DataFrame()

columnsList = ['in_ReportingUnitNativeID', 'geometry']
goutdf1 = pd.DataFrame(columns=columnsList, index=newgdf.index)

goutdf1['in_ReportingUnitNativeID'] = "SD" + newgdf["OBJECTID"].replace("", 0).fillna(0).astype(int).astype(str)  # in_ReportingUnitNativeID needs to match source from above equivlaent datframe
goutdf1['geometry'] = newgdf['geometry']
goutdf1 = goutdf1.drop_duplicates().reset_index(drop=True)
print(len(goutdf1))
goutdf1.head()

In [ ]:
#

#### Regulatory Area #2 shapefile info

In [ ]:
# etc etc...

#### Concatenate goutdf shapefile info into single output

In [ ]:
# Concatenate Basin & Subbasin DataFrames
frames = [goutdf1] # add geoutdf dataframes here
goutdf = pd.concat(frames).reset_index(drop=True)

print(len(goutdf))
goutdf.head()

## Export Data

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(outdf.dtypes)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(goutdf.dtypes)

In [ ]:
# Export out to CSV.
outdf.to_csv('RawInputData/Pre_sdMain.zip', compression=dict(method='zip', archive_name='Pre_Main.csv'), index=False)  # The output, save as a zip
goutdf.to_csv('RawInputData/P_Geometry.zip', compression=dict(method='zip', archive_name='P_Geometry.csv'), index=False)  # The output geometry.